# os
Este módulo fornece uma maneira simples de usar funcionalidades que são dependentes de SO como abrir e escrever em um arquivo, criar diretórios, etc

# bs4
Módulo que facilita a raspagem de dados de páginas web, já que faz o parser de HTML ou XML e permite iterar, pesquisar e modificar o conteúdo de uma página web.

# requests
Módulo que permite enviar requisições HTTP de maneira fácil

# pathlib
Módulo com várias classes para lidar com os caminhos de sistema

# pandas
Pacote que fornece estruturas de dados rápidas e flexíveis para lidar com dados estruturados e relacionados

# math
Biblioteca que fornece várias funções matemáticas

In [5]:
#Baixando dependências
import os
from bs4 import BeautifulSoup
import requests
import pathlib
import pandas as pd
import math

- os.path.abspath('') #retorna o caminho até o diretório do script que usa essa função
- os.makedirs(PATH_REPOSITORIO, exist_ok=True) #cria (ou não, caso o repositório já exista) um repositório com o path passado 

In [8]:
#Preparando repositório para guardar arquivo bruto baixado pela Internet e outro para guardar o arquivo após ser tratado
link = "https://www.epe.gov.br/pt/publicacoes-dados-abertos/publicacoes/BEN-Series-Historicas-Completas"
path_raiz =  os.path.abspath('')
path_input =  path_raiz + '/Input/'
path_output =  path_raiz + '/Output/'
os.makedirs(path_input, exist_ok=True)
os.makedirs(path_output, exist_ok=True)

- headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0'
}

soup = BeautifulSoup(requests.get(LINK, headers=headers).content, 'html.parser') # cria um objeto da classe BeautifulSoup, definindo o site e o tipo de parser

- soup.select(TAG) # retorna uma lista com todas tags do tipo TAG presentes na página web

- str(tag.encode_contents(), encoding='utf-8').replace('\n', '') #retorna o html interno dentro da tag

URL para colocar antes do link procurado = 'https://www.epe.gov.br'

In [9]:
#Obtendo o link para baixar o arquivo da Internet
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0'
}
soup = BeautifulSoup(requests.get(link, headers=headers).content, 'html.parser')
for tag in soup.select('a'):
    html_interno = str(tag.encode_contents(), encoding='utf-8').replace('\n', '')
    if html_interno.find("Matrizes Consolidadas") != -1:
            link_baixa_arquivo = 'https://www.epe.gov.br' + tag['href']
            break

- requests.get(LINK_DO_ARQUIVO_A_SER_BAIXADO, allow_redirects=True) #Faz uma requisição GET para o conteúdo do arquivo do link e retorna uma Response

- with open(NOME_ARQUIVO, 'wb') as output:
    output.write(RESPONSE) #Cria ou abre o arquivo NOME_ARQUIVO e escreve o resultado da requisição GET nesse arquivo

- pathlib.Path(PATH_ATUAL_ARQUIVO).rename(NOVO_PATH_ARQUIVO) # transporta o arquivo no caminho PATH_ATUAL_ARQUIVO para o NOVO_PATH_ARQUIVO

In [12]:
#Baixando o arquivo e guardando na pasta Input
r = requests.get(link_baixa_arquivo, allow_redirects=True)
nome_arquivo = "Input BEN.xlsx" 
with open(nome_arquivo, 'wb') as output:
    output.write(r.content)
pathlib.Path(path_raiz + '/' + nome_arquivo).rename(path_input + nome_arquivo)

PosixPath('/home/victorio/Documents/dojoEquipe04/Input/Input BEN.xlsx')

- titulos = ['CONTA','PRODUÇÃO','IMPORTAÇÃO']

- pd.DataFrame() #Cria um Data Frame vazio

- for sheet_name, df in pd.read_excel(PATH_EXCEL, sheet_name=None).items() #Itera por todas as planilhas presentes do arquivo Excel localizado no caminho PATH_EXCEL

- DF.drop(ROTULO, axis=EIXO, inplace=True) #Apaga uma linha (padrão) ou coluna (axis=1) com base no rótulo da mesma, o atributo inplace faz com que a coluna seja apagada no DataFrame que chamou o método

- DF.dropna(inplace=True) #Apaga as linhas (padrão) ou coluna (axis=1) que possuem um NaN

- len(DATA_FRAME) #retorna o número de linhas de um DATA_FRAME

- DF.drop(index=df.iloc[LINHA].name, inplace=True) #Apaga a linha com posição LINHA

- DF.transpose() #inverte um Data Frame, as linhas viram colunas e vice-versa

- [NUMERO_REPITIDO] * NUM_LINHAS #cria um Series que pode se tornar o conteúdo de uma nova coluna do Data Frame]

- pd.concat([DATA_FRAME_TOTAL, DATA_FRAME]) #concatena o DATA_FRAME abaixo do DATA_FRAME_TOTAL

- DF.iloc[LINHA_INICIO:LINHA_FIM,COLUNA_INICIO:COLUNA_FIM].applymap(lambda x : int(x)) #percorre todos os elementos do Data Frame no intervalo do Data Frame e transforma eles em inteiros

- DF.to_csv(PATH_ARQUIVO_SERA_SALVO, index=False, header=False) #salva o Data Frame como um arquivo csv no caminho PATH_ARQUIVO_SERA_SALVO


In [13]:
#Tratando tabela bruta
titulos = ['CONTA','PRODUÇÃO','IMPORTAÇÃO']
cont = 0
df_total = pd.DataFrame()
for sheet_name, df in pd.read_excel(path_input + nome_arquivo, sheet_name=None).items():
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.dropna(inplace=True)
    i = 0
    while len(df.index) != 3:
        titulo = df.iloc[i, 0]
        titulo = titulo.replace(' ', '')
        if titulo not in titulos:
            df.drop(index=df.iloc[i].name, inplace=True)
        else:
            i += 1
    df = df.transpose()
    num_rows = len(df)
    new_column_data = [sheet_name] * num_rows
    df['ANO'] = new_column_data
    if cont:
        df.drop(index=df.iloc[0].name, inplace=True)
    cont = 1
    df_total = pd.concat([df_total, df])
df_total.iloc[0,0] = 'FONTE'   
df_total.iloc[0, -1] = 'ANO'
#df_total.iloc[1:, 1:-1] = df_total.iloc[1:, 1:-1].applymap(lambda x : int(x))
df_total.iloc[1:, 1:-1] = df_total.iloc[1:, 1:-1].applymap(lambda x : math.floor(abs(x)) if (abs(x) * 10 - int(abs(x)) * 10) < 5 else math.ceil(abs(x)))
df_total.to_csv(path_output + "BEN tratado.csv", index=False, header=False)
print(df_total)

                                                 11              12  \
Unnamed: 1                                    FONTE      PRODUÇÃO     
Unnamed: 2                                PETRÓLEO           156398   
Unnamed: 3                             GÁS NATURAL            49971   
Unnamed: 4                            CARVÃO VAPOR             2311   
Unnamed: 5                      CARVÃO METALÚRGICO                0   
...                                             ...             ...   
Unnamed: 24         OUTRAS SECUNDÁRIAS DE PETRÓLEO                0   
Unnamed: 25   PRODUTOS NÃO ENERGÉTICOS DE PETRÓLEO                0   
Unnamed: 26                               ALCATRÃO                0   
Unnamed: 27               ENERGIA SECUNDÁRIA TOTAL                0   
Unnamed: 28                                 TOTAL             49629   

                           13   ANO  
Unnamed: 1       IMPORTAÇÃO     ANO  
Unnamed: 2              12724  2022  
Unnamed: 3               7722  20